In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import findspark
findspark.init()

In [ ]:
spark = SparkSession.builder.appName("lets do some querying").getOrCreate()
spark.conf.set("spark.sql.debug.maxToStringFields", 1000)

In [ ]:
df = spark.read.parquet(f"/parkingviolations/raw_all.parquet")
df.cache()

In [ ]:
# Group by plate_id and count the occurrences
violations_count = df.groupBy("plate_id").count()

# Order by count in descending order and select the top 10
top_violated_plates = violations_count.orderBy("count", ascending=False).limit(10)

# Show the result
top_violated_plates.show()
